In [435]:
# general
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

# first used in exercise two
import pca
from sklearn import preprocessing # for scale

# first used in exercise three
import coorddescent as cd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

In [137]:
import importlib
importlib.reload(pca)

<module 'pca' from '/Users/andrewenfield/work/github/Data558/Week05/pca.py'>

# Exercise One

**TODO**

# Exercise Two

*"(a) Generate a simulated data set with 20 observations in each of three classes (i.e. 60 observations total), and 50 variables. Hint: There are a number of functions in numpy that you can use to generate data. One example is the numpy.random.normal() func- tion; numpy.random.uniform() is another option. Be sure to add a mean shift to the observations in each class so that there are three distinct classes."*

A quick note: I had a hard time getting the algorithm to converge when I used very small mean shifts of 0.5 - I tried a lot of different step size value combinations and got nothing but non-convergence. I was able to get it to converge with a larger mean shift, so I'm going with that larger mean shift here.

In [201]:
from sklearn.datasets import fetch_lfw_people

faces = fetch_lfw_people(min_faces_per_person=30) 
print(faces.target_names)  # Images of 34 different people
print(faces.images.shape)  # 2370 images, each of which is 62x47 pixels
print(faces.data.shape)    # Each image is unravelled to a vector of size 2914

['Alejandro Toledo' 'Alvaro Uribe' 'Andre Agassi' 'Ariel Sharon'
 'Arnold Schwarzenegger' 'Colin Powell' 'David Beckham' 'Donald Rumsfeld'
 'George W Bush' 'Gerhard Schroeder' 'Gloria Macapagal Arroyo'
 'Guillermo Coria' 'Hans Blix' 'Hugo Chavez' 'Jacques Chirac'
 'Jean Chretien' 'Jennifer Capriati' 'John Ashcroft' 'John Negroponte'
 'Junichiro Koizumi' 'Kofi Annan' 'Laura Bush' 'Lleyton Hewitt'
 'Luiz Inacio Lula da Silva' 'Megawati Sukarnoputri' 'Nestor Kirchner'
 'Recep Tayyip Erdogan' 'Roh Moo-hyun' 'Serena Williams'
 'Silvio Berlusconi' 'Tom Ridge' 'Tony Blair' 'Vicente Fox'
 'Vladimir Putin']
(2370, 62, 47)
(2370, 2914)


In [202]:
mean_vals = [0, 10, 20]
np.random.seed(42)
d = pd.DataFrame(np.vstack([np.random.normal(mean, 
                        size=(20,50)) for mean in mean_vals]))
d.insert(0, 'Class', np.repeat(['A','B','C'], 20))
d[:1]

,Class,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,A,0.496714,-0.138264,0.647689,1.52303,-0.234153,-0.234137,1.579213,0.767435,-0.469474,...,0.738467,0.171368,-0.115648,-0.301104,-1.478522,-0.719844,-0.460639,1.057122,0.343618,-1.76304


In [203]:
d_values = d.values[:, 1:51].astype('float')
d_values_centered = preprocessing.scale(d_values, with_std=False)
d_values_centered[0]

array([ -9.47652431, -10.36087989,  -9.29538686,  -8.58559277,
       -10.16925639, -10.28390443,  -8.43284888,  -9.41223556,
       -10.46780705,  -9.85403586, -10.39694444, -10.58329323,
        -9.84789351, -11.77688128, -12.0374925 , -10.61884412,
       -11.00006027,  -9.6519797 , -10.75006323, -11.58583051,
        -8.81179199, -10.29558276,  -9.91366629, -11.39948789,
       -10.70784122,  -9.85670811, -11.13090521,  -9.8084079 ,
       -10.73701623, -10.46793313, -10.56025456,  -7.99222798,
        -9.95515398, -11.09243514,  -9.20584621, -11.29344291,
        -9.73924429, -11.91219899, -11.31890736,  -9.77094352,
        -9.32654553,  -9.90398641, -10.16759232, -10.2470722 ,
       -11.37186667, -10.72705805, -10.23186415,  -8.66765149,
        -9.90986555, -11.76248424])

In [258]:
#X = faces.data
#eta_0, t_0, epoch_count = .00001, 1, 50

X = d_values
eta_0, t_0, epoch_count = 0.001, 2, 100

*"(b) Run your own normalized Oja algorithm on the 60 observations. You should try to implement the algorithm yourself, but you may look back at the labs if you need help. Plot the first two principal component score vectors. Compare your results to the ones obtained with scikit-learn’s PCA algorithm. Use a different color to indicate the observations in each of the three classes. If the three classes appear separated in this plot, then you’re done and you can launch the computations on AWS. If not, then return to part (a) and modify the simulation so that there is greater separation between the three classes."*

In [259]:
#bar = pca.oja_fit(d_values_centered_copy, 1, 0.001, 2, 100)
#bar[0]

In [260]:
pca_sklearn = PCA(5, svd_solver='randomized')
pca_sklearn.fit(X)
print(pca_sklearn.components_[0])
print(pca_sklearn.components_[1])

[-0.14233391 -0.14325963 -0.1412039  -0.13801734 -0.13977184 -0.14364433
 -0.13991273 -0.14104493 -0.14228272 -0.14028026 -0.13925715 -0.14081765
 -0.1409513  -0.13850097 -0.1433206  -0.14209824 -0.13842943 -0.13647746
 -0.14296263 -0.14078382 -0.1406264  -0.14656184 -0.13859031 -0.14020859
 -0.14084239 -0.14366725 -0.14535678 -0.14170943 -0.14213593 -0.14141145
 -0.14192793 -0.14247393 -0.14370843 -0.14302147 -0.13910328 -0.14296752
 -0.14035279 -0.14268592 -0.13854042 -0.14386981 -0.14474806 -0.13853575
 -0.14457845 -0.13896695 -0.14065459 -0.14347999 -0.1421975  -0.13715082
 -0.14139857 -0.14340664]
[-0.1483044  -0.15649313 -0.02742337  0.05236321  0.09140626  0.21919547
  0.04730121  0.05943288  0.18635766  0.07401636 -0.14690844  0.10929665
 -0.21394577 -0.23236245  0.08351255  0.00858012  0.22214401  0.03630085
  0.00875048 -0.06700667  0.24595039  0.15736187  0.08113937 -0.00670334
 -0.25491227  0.06126596 -0.00613763  0.02599831 -0.23115294 -0.25220704
  0.02516941  0.16921129 

In [307]:
eta_0, t_0, epoch_count = 1, 5, 1000

Z = X - np.mean(X, axis=0)  # center the data

a_0 = np.random.randn(np.size(Z, 1))  # starting point
a_0 /= np.linalg.norm(a_0, axis=0)
v1, _ = pca.oja(copy.deepcopy(Z), a_0, eta_0, t_0, epoch_count)
print(v1)

Z1 = pca.deflate(Z, v1)
v2, _ = pca.oja(copy.deepcopy(Z1), a_0, eta_0, t_0, epoch_count)
print(v2)

[ 0.14421688  0.14524653  0.14241444  0.13975037  0.13780312  0.14393627
  0.13921924  0.14171545  0.14215492  0.13966872  0.14078557  0.14019853
  0.13994375  0.13713278  0.14222145  0.14173678  0.13944096  0.1363648
  0.14256834  0.13915115  0.14030459  0.14307384  0.13738016  0.14058151
  0.13839918  0.14190638  0.14703884  0.14007877  0.1443727   0.14048751
  0.14488143  0.14326805  0.14336822  0.14223922  0.13871945  0.14183431
  0.13919763  0.14204785  0.13785358  0.14692811  0.14436192  0.1392139
  0.1462009   0.13883138  0.14346422  0.14109078  0.14418752  0.13692471
  0.14111302  0.14481343]
[  1.45633968e-01   1.37666896e-01   6.70423107e-02  -2.17624208e-02
  -1.02138526e-01  -1.94312731e-01  -7.88557436e-02  -6.84307157e-02
  -1.99798496e-01  -4.71073119e-02   1.63968172e-01  -8.54023227e-02
   2.04055067e-01   2.23867295e-01  -1.02037037e-01  -1.96363611e-02
  -2.22652863e-01  -2.41069615e-02  -1.49006976e-02   4.78313505e-02
  -2.44106196e-01  -1.69993192e-01  -9.64223103

In [180]:
pca_sklearn = PCA(50, svd_solver='randomized')
pca_sklearn.fit(d_values_centered)

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=False)

In [183]:
pca_sklearn.components_[1]

array([-0.14820248, -0.15577997, -0.02747821,  0.05217483,  0.09001604,
        0.21697314,  0.04838229,  0.0579049 ,  0.1868853 ,  0.07290217,
       -0.1472483 ,  0.10724179, -0.21673595, -0.23128117,  0.08330134,
        0.00877515,  0.22128086,  0.03809422,  0.00873939, -0.0690226 ,
        0.24575393,  0.15597047,  0.08329307, -0.00890795, -0.25341302,
        0.06472497, -0.00552899,  0.02767339, -0.23068174, -0.2512671 ,
        0.02564334,  0.17065277,  0.19201366, -0.20402114,  0.12081652,
       -0.03829716,  0.02052802, -0.00479948, -0.13600818, -0.20458192,
        0.02034034, -0.00940222, -0.13363653,  0.12899377, -0.3380589 ,
        0.06794979,  0.09806946,  0.16235379,  0.01031238,  0.02908066])

# Exercise Three

*"(a) Use the numpy.random.normal() function to generate a predictor X of length n = 100, as well as a noise vector ε of length n = 100."*

In [308]:
X = np.random.normal(size=100)
e = np.random.normal(size=100)
(X.shape, e.shape)

((100,), (100,))

*"(b) Generate a response vector Y of length n = 100 according to the model Y =β0 +β1X+β2X2 +β3X3 +ε,
where β0, β1, β2, and β3 are constants of your choice. Here superscripts denote powers of X."*

I'm just choosing some coefficients.

In [309]:
b0 = 27
b1 = 11
b2 = 5
b3 = 3

In [310]:
y = b0 + (b1*X) + (b2*(X**2)) + (b3*(X**3)) + e
y.shape

(100,)

In [311]:
# and we'll make X a 100,1 array rather than a vector,
# for better use w/ sklearn functions
X = X[:, np.newaxis]
X.shape

(100, 1)

And we'll scale X and center y. 

In [322]:
X_scaled = preprocessing.scale(X)
y_centered = preprocessing.scale(y, with_std=False)

*"(c) Now fit a LASSO model to the simulated data using your own coordinate descent algorithm. Use X, X2, . . . , X10 as predictors. Set λ = 1."*

Note that I found with lambda = 1 that I didn't get close convergence for all coefficients: although some of the higher coefficients had numbers didn't jump around a huge amount, they did jump around some. That said, the lower coefficients - the first four - stayed pretty constant (and the fifth and six were zeroed out by the algorithm). (This held regardless of whether I bumped up the iterations to something like 5000.)

In [405]:
X_scaled_poly10 = PolynomialFeatures(degree=10, include_bias=False).fit_transform(X_scaled)
X_scaled_poly10.shape

(100, 10)

In [406]:
n = np.size(X_scaled_poly10, 0)
beta_init = np.zeros(np.size(X_scaled_poly10, 1))
max_iter = 1000

In [407]:
betas_rand = cd.randcoorddescent(beta_init, X_scaled_poly10, y_centered, 1, max_iter)
betas_rand[-1, :]

array([  1.10616632e+01,  -4.47876250e+00,   1.85917711e+00,
         3.58648303e+00,   2.25145468e-01,  -2.23217761e-01,
         1.10173283e-01,  -5.57980881e-02,  -2.43346606e-02,
         3.29610103e-03])

*"(d) Select the optimal value of λ on a held-out validation set using your own coordinate descent algorithm. Create plots of the error on the validation set as a function of λ for your own coordinate descent algorithm and scikit-learn’s LASSO algorithm; use different colors for the two curves. Discuss the results obtained.*

**TODO** Update to use a held out validation set.

In [444]:
def mse_from_beta_coefs(beta_coefs):
    return mean_squared_error(y_centered, X_scaled_poly10.dot(beta_coefs))

In [426]:
lambdas = [10**k for k in range(-8, 5)]
lambdas

[1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

In [434]:
def mse_for_lambda(lam):
    beta_init = np.zeros(np.size(X_scaled_poly10, 1))
    betas_rand = cd.randcoorddescent(beta_init, X_scaled_poly10, y_centered, lam, max_iter)
    return(mse(betas_rand[-1, :]))

mses = [mse_for_lambda(lam) for lam in lambdas]
list(zip(lambdas, mses))

[(1e-08, 6.3197201660133908),
 (1e-07, 6.1828994789615956),
 (1e-06, 6.168319195823913),
 (1e-05, 6.2765664510526697),
 (0.0001, 6.2925517495095766),
 (0.001, 6.2682356826539616),
 (0.01, 6.3688047545359883),
 (0.1, 6.1631304155488298),
 (1, 6.3526716279949245),
 (10, 6.5772137511579754),
 (100, 8.6874435817789113),
 (1000, 52.436675675985626),
 (10000, 121.45884410180601)]

In [450]:
# TODO run for all values of lambda, but do this after adding in the hold out set
results = Lasso(alpha=0.0001, fit_intercept=False, max_iter=10000000).fit(X_scaled_poly10, y_centered)
print(results.coef_)
print(mse_from_beta_coefs(results.coef_))

[  9.46811929 -15.38790745   6.65491137  25.1143633   -3.76599474
 -12.61430244   1.30994699   2.71492084  -0.14189205  -0.21004894]
4.62506779495


**TODO** plot

In [408]:
X_scaled_poly10.shape

(100, 10)

In [409]:
y_centered[:1], X_scaled_poly10[:1]

(array([-8.56285377]),
 array([[ -4.49690062e-01,   2.02221152e-01,  -9.09368425e-02,
           4.08933944e-02,  -1.83893531e-02,   8.26950932e-03,
          -3.71871616e-03,   1.67226970e-03,  -7.52003067e-04,
           3.38168306e-04]]))

In [410]:
betas_rand[-1, :]

array([  1.10616632e+01,  -4.47876250e+00,   1.85917711e+00,
         3.58648303e+00,   2.25145468e-01,  -2.23217761e-01,
         1.10173283e-01,  -5.57980881e-02,  -2.43346606e-02,
         3.29610103e-03])

In [411]:
betas_rand[-1, :].shape

(10,)

In [412]:
sum(betas_rand[-1, :] * X_scaled_poly10[10])

7.527854935512817

In [413]:
X_scaled_poly10.dot(betas_rand[-1, :])

array([ -5.90889455e+00,  -6.59896247e+00,  -2.82858900e-02,
        -3.72666737e+00,  -1.22388019e+00,   4.62496037e+01,
         5.99943352e-01,   8.14071406e+00,  -8.68521743e+00,
         1.02360082e+01,   7.52785494e+00,   5.60957266e-01,
        -9.22993701e+00,  -9.26681890e+00,   3.61218042e+00,
        -3.96936653e+01,   1.66572692e+00,   9.53784833e+00,
        -1.99417247e+00,  -4.13004096e-01,   5.76691653e+00,
         4.41397353e+01,   1.94973859e+00,   6.35914625e+01,
        -2.57090507e+01,   3.58255065e+01,  -4.49204498e+01,
         4.94963945e+00,  -2.21245623e+01,   6.38931558e+00,
         9.15314550e-01,  -3.40063044e+00,   1.61835960e+00,
        -2.11095743e+00,   9.55435981e+00,  -1.68364729e+01,
        -2.59540752e+00,   7.66347003e+00,   4.58738789e+00,
         3.84815246e+00,   2.77141811e+01,   1.23209087e+00,
        -1.01811619e+01,  -3.50056125e+01,   4.43551445e-01,
        -9.23715909e+00,  -2.36589573e+01,   2.58187221e+01,
         5.70266044e+01,

In [414]:
mean_squared_error(y_centered, X_scaled_poly10.dot(betas_rand[-1, :]))

6.1833193555900099

In [416]:
mse(betas_rand[-1, :])

6.1833193555900099

*"(e) Now generate a response vector Y according to the model Y = β0 + β7X7 + ε. Fit a LASSO model using your own coordinate descent algorithm and scikit-learn’s LASSO algorithm. Discuss the results obtained."*

# Exercise Four

**TODO** Something and note here.

# Exercise Five

**TODO** Possibly just refer to what I submitted last week, unless something's changed.

* Is there a concept you would love to see covered during the course?
* Is there a concept that has been used multiple times during the course, but you still have no idea what it means?
* Do you feel that you learn better when the course material is covered using slides, or tablet/whiteboard, or numerical illustrations?